# 6.4: Resampling Exercises

## Getting Started

### Import Libraries 

We import our standard libraries and specific objects/libraries at the top level of our notebook.

In [23]:
# Load our previous libraries and objects
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import train_test_split

# Import new libraries and objects
from sklearn.model_selection import cross_validate
from ISLP.models import sklearn_sm

import warnings
warnings.filterwarnings('ignore') # mute warning messages

## The Validation Set Approach

We explore the use of the validation set approach in order to estimate the test error rates that result from fitting various linear models on the `Auto` data set.

We use the function `train_test_split()` to split the data into training and validation sets. As there are 392 observations, we split into two equal sets of size 196 using the argument `test_size=196`. We want to set a random seed (`random_state=0`) so that the random results can be reproduced again.

In [43]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,
                                         test_size=196, # split in two
                                         random_state=4) # random seed

In [44]:
for a,b in zip(Auto_train['mpg'], Auto_valid['mpg']):
    if a == b:
        print(a,b)

18.0 18.0
20.5 20.5
15.0 15.0
22.0 22.0


In [48]:
len(Auto['mpg']) - len(set(Auto['mpg']))

265

In [50]:
np.power.outer(2,3)

8

Now we can fit a linear regression using only the observations corresponding to the training set `Auto_train`.

In [25]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

We now use the `predict()` method of results evaluated on the model matrix for this model created using the validation data set. We also calculate the validation MSE of our model.

In [10]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

20.755407959228602

So our estimated test MSE for the linear regression is 23.62.

**What would happen if we used a different set of observations for our training set? Repeat the above
process using a different seed (i.e. set.seed(2)) and report the differences.**

## Leave-One-Out Cross-Validation

The simplest way to cross-validate in Python is to use `sklearn`. The `ISLP` package provides a wrapper (`sklearn_sm()`), that allows us to easily use the cross-validation tools of `sklearn` with models fit by `statsmodels`.

In [ ]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

The arguments to `cross_validate()` are: an object with the appropriate `fit()`, `predict()`, and `score()` methods, an array of features `X` and a response `Y`. We also included an additional argument `cv` to `cross_validate()`; specifying an integer 
$K$ results in $K$-fold cross-validation.

We can repeat this procedure for increasingly complex polynomial fits. To automate the process, we again use a for loop which iteratively fits polynomial regressions of degree 1 to 5, computes the associated cross-validation error, and stores it in the $i$th element of the vector `cv_error`.

**Write a for loop that fits the Auto data using polynomials up to degree 5. Compute the LOOCV
test error rate for each of them. Which degree polynomial fits the data the best according to
the test error rate and the bias-variance trade-off?**

Tip: use `np.power.outer` function to construct `X`

In [14]:
## Your codes
y_train

357    24.2
94     12.0
218    33.5
355    30.7
292    34.1
       ... 
256    20.6
131    16.0
249    20.2
152    15.0
362    28.0
Name: mpg, Length: 196, dtype: float64

In [18]:
Auto_train

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
357,24.2,6,146.0,120,2930,13.8,81,3,datsun 810 maxima
94,12.0,8,455.0,225,4951,11.0,73,1,buick electra 225 custom
218,33.5,4,85.0,70,1945,16.8,77,3,datsun f-10 hatchback
355,30.7,6,145.0,76,3160,19.6,81,2,volvo diesel
292,34.1,4,86.0,65,1975,15.2,79,3,maxda glc deluxe
...,...,...,...,...,...,...,...,...,...
256,20.6,6,231.0,105,3380,15.8,78,1,buick century special
131,16.0,6,250.0,100,3781,17.0,74,1,chevrolet chevelle malibu classic
249,20.2,8,302.0,139,3570,12.8,78,1,mercury monarch ghia
152,15.0,6,250.0,72,3432,21.0,75,1,mercury monarch


In [21]:
valid_pred

352    24.167645
16     24.650897
288    17.402115
281    25.778485
201    28.839082
         ...    
170    28.839082
29     26.100653
162    22.556804
149    29.483418
366    26.744989
Length: 196, dtype: float64

*These exercises were adapted from :* James, Gareth, et al. An Introduction to Statistical Learning: with Applications in Python, Springer, 2023.